In [2]:
import pyvisa as visa
#import visa
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt
import datetime

N = 100000 # minimum digit 10uA

#Check 6221 compliance, Check 2400 output off state, very important!!!!

In [3]:
import datetime
print(datetime.datetime.now())

2022-05-23 15:26:38.245256


In [4]:
rm = visa.ResourceManager()
rm.list_resources()

('ASRL1::INSTR',
 'GPIB0::15::INSTR',
 'GPIB1::1::INSTR',
 'GPIB1::7::INSTR',
 'GPIB1::8::INSTR',
 'GPIB1::12::INSTR',
 'GPIB1::13::INSTR',
 'GPIB1::23::INSTR')

In [191]:
#volt.write("*RST")

(6, <StatusCode.success: 0>)

In [4]:
#switch
ac_6221_id = 12
dc_2400_id = 23
nanovolt_id = 1

ac = rm.open_resource('GPIB1::{}::INSTR'.format(ac_6221_id))
dc = Keithley2400("GPIB1::{}".format(dc_2400_id))
volt = rm.open_resource('GPIB1::{}::INSTR'.format(nanovolt_id))

In [ ]:
#volt.write(":sens:volt:chan2:rang 0.8")

In [5]:
def trigger_ac(ac, offs=0.005, amp=1e-6, duration=0.005, wait_after_arm=2):
    ac.write("SOUR:WAVE:ABOR")
    ac.write("SOUR:WAVE:OFFS {}".format(offs))
    ac.write("SOUR:WAVE:AMPL {}".format(amp))
    ac.write("SOUR:WAVE:DUR:TIME {}".format(duration))
    ac.write("SOUR:WAVE:ARM")
    time.sleep(wait_after_arm)
    ac.write("SOUR:WAVE:INIT")

In [6]:
def init_switch_communication(ac=ac, dc=dc, volt=volt):
    
    dc.apply_current()# Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = 100e-6            # Sets the source current to 0.1 mA
    dc.enable_source() 
    #print("enable dc1 current now, current:" + str(dc))
    print("check nano volt 1:")
    
    for i in range(10):
        if i > 6:
            print(volt.query("fetch?"))
        time.sleep(0.1)
        
    time.sleep(1)
    dc.shutdown()   
    print("dc1 current disabled")
    
    print("check ac_1: offerset 50uA, amp1uA, duration 3s")
    trigger_ac(ac, offs=0.00005, amp=1e-6, duration=3, wait_after_arm=2)
    time.sleep(1)


In [7]:
init_switch_communication()

check nano volt 1:
+9.81433832E-06

+9.81433832E-06

+9.81433832E-06

dc1 current disabled
check ac_1: offerset 50uA, amp1uA, duration 3s


In [40]:
init_communication()

check ac_1: offerset 50uA, amp1uA, duration 10s
check ac_2: offerset 50uA, amp1uA, duration 10s
check ac_mid: offerset 50uA, amp1uA, duration 10s
enable dc current now, current:<pymeasure.instruments.keithley.keithley2400.Keithley2400 object at 0x0000000004BE8F60>
check nano volt:
-2.80284775E-07

-8.83834280E-05

-8.83774182E-05

-8.83972507E-05

-8.83089059E-05

-8.82824626E-05

-8.83954477E-05

-8.83581867E-05

-8.82878715E-05

-8.82091425E-05

dc current disabled


In [8]:
def measure_resistance(dc, volt, dc_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.1):
    dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up = []
    for i in range(30):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up.append(float(volt.query("fetch?")))
    v_up = sorted(v_up)
    v_up = v_up[3: -3]
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down = []
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down.append(float(volt.query("fetch?")))
    
    dc.shutdown() 
    #dc.source_current = dc_current
    #dc.enable_source() 
    
    v_down = sorted(v_down)
    v_down = v_down[3: -3]
    
    average_v = (np.array(v_up).mean() - np.array(v_down).mean())/2
    average_v = np.array(v_up).mean()
    std_v = (np.array(v_up).std() + np.array(v_down).std())/2
    std_v = np.array(v_up).std()
    return average_v, std_v

In [9]:
def generate_current(currents, steps, reversed=False):
    res = []
    currents = [int(i * N) for i in currents]
    steps = [int(i * N) for i in steps]
    #res += list(range(0, currents[0], steps[0]))  
    assert (len(currents) == len(steps))
    for i in range(len(currents)):
        res.extend([f for f in range(currents[i], currents[i + 1] if i+1 < len(currents) else 0, 
                                     -1 * steps[i])])
    res = res + [0] + [-1 * f for f in res[::-1]]
    res += res[::-1][1:]
    res = list(range(0, currents[0], steps[0])) + res
    if reversed:
        return [-1* i / N for i in res] 
    else:
        return [i / N for i in res] 


In [10]:
def switch_measurement(sample_id='', currents=[], dc_current=100e-6, duration=0.001, pulse_count=2,
                        wait_after_trigger=15, wait_after_hall=2, dc=dc, volt=volt, ac=ac):
    file_name = "data/{}_{}ms_wait{}s_{}.csv".format(sample_id,  
                                                                   int(duration * 1e3), 
                                                                   wait_after_trigger, 
                                                                   int(time.time()))
    with open(file_name, 'w') as f:
        f.write("Pulse_I,average_v, std_v, dc_current\n")

    for i in currents:
        for _ in range(pulse_count):
            #print("current pulse: {}mA".format(i*1000))
            trigger_ac(ac, offs=i, duration=duration)
            time.sleep(wait_after_trigger)
            average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
            with open(file_name, 'a') as f:
                f.write("{},{},{},{}\n".format(i, average_v, std_v, dc_current))
            print("current pulse: {}mA -> Rxy: {}mV".format(i*1000, average_v*1000))
            time.sleep(wait_after_hall)


In [23]:
do_task_single = True
current = [5e-3]
steps = [10e-3]
currents = generate_current(current, steps, True)
print(currents)

[0.0, -0.005, 0.0, 0.005, 0.0, -0.005]


In [22]:
# single measurement
if not do_task_single:
    raise

switch_measurement(sample_id='CoFeGd_Grad_40Oe', currents=currents, dc_current=100e-6, duration=0.001,
                   wait_after_trigger=0.2, wait_after_hall=0.2, dc=dc, volt=volt, ac=ac,pulse_count=1)

do_task_single = False


current pulse: 0.0mA -> Rxy: 0.009233611374999999mV
current pulse: -10.0mA -> Rxy: 0.009243680102857144mV
current pulse: -20.0mA -> Rxy: 0.009226663131428572mV
current pulse: -30.0mA -> Rxy: 0.009716046399285715mV
current pulse: -40.0mA -> Rxy: -0.015870143235714285mV
current pulse: -50.0mA -> Rxy: -0.07594743595714284mV
current pulse: -40.0mA -> Rxy: 9.899999999999999e+40mV
current pulse: -30.0mA -> Rxy: 9.899999999999999e+40mV


KeyboardInterrupt: 

In [1]:
def generate_max_c(max_c, min_c, steps):
    res = list(range(int(min_c*N ), int(max_c*N ), int(steps * N)))
    return [i/ N  for i in res] + [max_c]

In [4]:
do_task_muti = True
max_c_max = 10e-3
min_c_max = 5e-3
max_steps = 2e-3
measure_steps = [1e-3]
max_currents = generate_max_c(max_c_max, min_c_max, max_steps)
print(max_currents)

[0.005, 0.007, 0.009, 0.01]


In [7]:
# muti measurement
if not do_task_muti:
    raise

for max_current in max_currents:
    current = [max_current]
    currents = generate_current(current, measure_steps)
    print(currents)
    continue
    switch_measurement(sample_id='test', currents=currents, dc_current=100e-6, duration=0.001,
                   wait_after_trigger=15, wait_after_hall=2, dc=dc, volt=volt, ac=ac)

do_task_muti = False



[0.0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.004, 0.003, 0.002, 0.001, 0.0, -0.001, -0.002, -0.003, -0.004, -0.005, -0.004, -0.003, -0.002, -0.001, 0.0, 0.001, 0.002, 0.003, 0.004, 0.005]
[0.0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.006, 0.005, 0.004, 0.003, 0.002, 0.001, 0.0, -0.001, -0.002, -0.003, -0.004, -0.005, -0.006, -0.007, -0.006, -0.005, -0.004, -0.003, -0.002, -0.001, 0.0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007]
[0.0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.008, 0.007, 0.006, 0.005, 0.004, 0.003, 0.002, 0.001, 0.0, -0.001, -0.002, -0.003, -0.004, -0.005, -0.006, -0.007, -0.008, -0.009, -0.008, -0.007, -0.006, -0.005, -0.004, -0.003, -0.002, -0.001, 0.0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009]
[0.0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.009, 0.008, 0.007, 0.006, 0.005, 0.004, 0.003, 0.002, 0.001, 0.0, -0.001, -0.002, -0.003, -0.004, -0.005, -0.006, -0.007, -0.008, -0.009, -0.01